In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#---CONFIG FRAME---#
train_file = [
    "parsed_emg/subject_1/trial_1(right).csv",
    "parsed_emg/subject_1/trial_3(right).csv",
    "parsed_emg/subject_1/trial_5(right).csv",
    "parsed_emg/subject_1/trial_7(right).csv",
    "parsed_emg/subject_1/trial_9(right).csv",
]

test_file = "parsed_emg/subject_1/trial_11(right).csv"

label_col = 5
batch_size = 32
learning_rate = 0.01
epochs = 1
num_classes = 1

#---Data Loader---#
def load_csv(path):
    df = pd.read_csv(path, skiprows=1)
    df = df.drop(columns=df.columns[0], axis=1)
    X = df.iloc[:, 1].values.astype("float32").reshape(-1, 1)
    y = df.iloc[:, label_col].values.astype("int32")
    return X, y

#---Epoch Plotter---#



In [3]:
training_datasets = []
for f in train_file:
    X_tr, y_tr = load_csv(f)
    ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
    training_datasets.append(ds)

training_ds = training_datasets[0]
for ds in training_datasets[1:]:
    training_ds = training_ds.concatenate(ds)
training_ds = training_ds.shuffle(buffer_size=5000).batch(batch_size).prefetch(1)

X_test, y_test = load_csv(test_file)

model = tf.keras.Sequential([
    layers.Input(shape=(1,)),
    layers.Dense(
        num_classes,
        activation=None,
        kernel_regularizer=regularizers.l2(1e-4) 
    ),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss="categorical_hinge",
    metrics=["accuracy"],
)

model.fit(training_ds, epochs=epochs)

# Evaluate performance
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest loss: {test_loss:.4f} — Test accuracy: {test_acc:.4f}")



61121/61121 ━━━━━━━━━━━━━━━━━━━━ 91s 1ms/step - accuracy: 0.8998 - loss: 0.0537
15900/15900 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0883 - loss: 1.8519

Test loss: 1.2018 — Test accuracy: 0.2517


In [17]:
# Input a number (e.g., 0.09123)
input_number = 1203123123123.123

# Reshape to match model input shape (e.g., (batch_size, 1))
input_tensor = np.array([[input_number]])  # Shape (1, 1)

# Get prediction
prediction = model.predict(input_tensor)

print("Model Prediction:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Model Prediction: [[1.042276]]
